# The Gladius Data Model

### Data Model Structure


Collection: "clubs"

Document: "club_id"
Club data fields
Sub-collection: "events"
Document: "event_id"
Event data fields
Sub-collection: "occurrences"
Document: "occurrence_id"
Occurrence data fields
Sub-collection: "attendance"
Document: "attendance_id"
Field: "user_id" (to reference the attending user)
Field: "score" (record the score for the user at this occurrence)

Collection: "users"

Document: "user_id"
User data fields
Sub-collection: "scores"
Document: "score_id"
Field: "club_id" (to reference the club)
Field: "event_id" (to reference the event)
Field: "occurrence_id" (to reference the occurrence)
Field: "score" (the user's score for this event occurrence)


In [37]:
# Installs
#!pip install firebase-admin

In [2]:

# Import firebase_admin and initialize the app
import firebase_admin
from firebase_admin import auth, credentials, firestore
import random


Authentication

In [3]:
if not firebase_admin._apps:
    cred = credentials.Certificate("../firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)
# Get a reference to the firestore database
db = firestore.client()

# Create users with authentication

### Delete all auth users

In [4]:
user_list = auth.list_users()

# Iterate over each user and delete them
for user in user_list.users:
    auth.delete_user(user.uid)
    print(f"Deleted user: {user.uid}")

print("All users have been deleted.")

Deleted user: 7Z4T9MYgsANVCyiIrNeYc6qmekY2
Deleted user: 90i27gQkYrZBsKbkL5hDTXeoGlP2
Deleted user: AF5eBashD6hwModrtrAIY61t8t72
Deleted user: HMlWRnWByVMcm99HSMVSpYjSYPL2
Deleted user: W347Xw3QOUf1FVElyqK1G8ctcOU2
Deleted user: XLPZeIKmnGh5FbEGB9T3szbVYx32
Deleted user: nBcmsNgrDfYBJzEif1k2ZxNxwvr2
Deleted user: uy6mHpzUr3WalJv0O0YhL5t3qaf1
Deleted user: v4y1YsrLw0PYD6yZIazIwYrNADn2
Deleted user: vOqdKKZA0yY9QYVaUhCzs60lFXH3
Deleted user: yWVBs95uhYSrtum37zothgIHD9W2
All users have been deleted.


### Create new auth users

In [5]:
# Create a list of users to add
users = ["Mare", "Jüri", "Liisa", "Mart", "Evelin", "Tõnis", "Mari", "Henrik", "Triin", "Jaak", 'Bob']


user_uuids = {}
# Loop through the users and create them with a default password
for user in users:
  try:
    # Create the user with the email and password
    user_record = auth.create_user(
      email=f'{user.lower()}@example.com',
      password='123456'
    )
    user_uuids[user] = user_record.uid
    # Print the user ID
    print(f'Successfully created new user: {user_record.uid}')
  except firebase_admin.auth.AuthError as e:
    # Handle any errors
    print(f'Error creating new user: {e}')

Successfully created new user: 6r2Gikjj5aWaYisryYFjoBiJkzE2
Successfully created new user: oqvlcWzq4seCzNUGKZoXOjIuY2j2
Successfully created new user: 1lXewmV6wOQK39mCr0qKe0CO4gZ2
Successfully created new user: T3k81FzuaEXhykQ7NDRpbdHFZAc2
Successfully created new user: ft4HesNJ3rbi6j8mLDQSkweEha03
Successfully created new user: W3A36pW0VURcEwt3lFvtosfldA23
Successfully created new user: 1RBltFYi8hZ42ebVRTEwcEqVnJt2
Successfully created new user: TDhwPfb7Z8bVk0vDArGGkzQumGM2
Successfully created new user: 6SvkTL6XH8XSxSvEjTKFQNT3KW42
Successfully created new user: oVCgxSLLgYVQTrOjcSPxgr6YRar2
Successfully created new user: z579rpa8nuRusthIedgB7T1RlHM2


### Delete user collection

In [6]:
## Delete user collection
# Function to delete a collection and its documents
def delete_collection(collection_ref, batch_size):
    docs = collection_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        doc.reference.delete()
        deleted += 1

    if deleted >= batch_size:
        return delete_collection(collection_ref, batch_size)

# Specify the "users" collection to be deleted
users_collection_ref = db.collection("users")

# Delete the "users" collection and its documents
delete_collection(users_collection_ref, 500)

print("Users collection deleted successfully.")

Users collection deleted successfully.


## Add users collection

In [7]:
# Create a collection for users
users_collection = db.collection("users")

user_roles = {}

# Loop through the users and add them to the "users" collection
for user in users:
    age = random.randint(18, 60)
    email = f"{user.lower()}@example.com"
    occupation = random.choice(["Student", "Engineer", "Teacher", "Artist"])


for user in users:

    # Assign roles for each user in different clubs
    if user in ["Bob", "Mare"]:
        user_roles[user_uuids[user]] = {"club_id": "1", "role": "owner"}
    else:
        user_roles[user_uuids[user]] = {"club_id": "1", "role": "athlete"}

    user_data = {
        "name": user,
        "email": f"{user.lower()}@example.com",
        "age": random.randint(18, 60),
        "occupation": random.choice(["Student", "Engineer", "Teacher", "Artist"]),
        "is_active": random.choice([True, False]),
    }

    user_ref = users_collection.document(user_uuids[user])
    user_ref.set(user_data)
    print(f"Successfully created user: {user_uuids[user]}")

Successfully created user: 6r2Gikjj5aWaYisryYFjoBiJkzE2
Successfully created user: oqvlcWzq4seCzNUGKZoXOjIuY2j2
Successfully created user: 1lXewmV6wOQK39mCr0qKe0CO4gZ2
Successfully created user: T3k81FzuaEXhykQ7NDRpbdHFZAc2
Successfully created user: ft4HesNJ3rbi6j8mLDQSkweEha03
Successfully created user: W3A36pW0VURcEwt3lFvtosfldA23
Successfully created user: 1RBltFYi8hZ42ebVRTEwcEqVnJt2
Successfully created user: TDhwPfb7Z8bVk0vDArGGkzQumGM2
Successfully created user: 6SvkTL6XH8XSxSvEjTKFQNT3KW42
Successfully created user: oVCgxSLLgYVQTrOjcSPxgr6YRar2
Successfully created user: z579rpa8nuRusthIedgB7T1RlHM2


## Delete Club collection

In [8]:
## Delete user collection
# Function to delete a collection and its documents
def delete_collection(collection_ref, batch_size):
    docs = collection_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        doc.reference.delete()
        deleted += 1

    if deleted >= batch_size:
        return delete_collection(collection_ref, batch_size)

# Specify the "users" collection to be deleted
users_collection_ref = db.collection("clubs")

# Delete the "users" collection and its documents
delete_collection(users_collection_ref, 500)

print("Clubs collection deleted successfully.")

Clubs collection deleted successfully.


## Add club collection

In [9]:

# Create a collection for clubs
clubs = db.collection("clubs")

# Create a document for a club with some data fields
club_id = '1'
club_data = {
    "name": "Gladius Club",
    "description": "A Learn to Earn Club",
    "owners": [user_uuid for user_uuid, role in user_roles.items() if role["club_id"] == "1" and role["role"] == "owner"],
    "athletes": [user_uuid for user_uuid, role in user_roles.items() if role["club_id"] == "1"],

}
club_ref = clubs.document(club_id)
club_ref.set(club_data)


update_time {
  seconds: 1687545694
  nanos: 172232000
}

## Add events to clubs

### Listing events from calendar

In [25]:
googleCalendarId = "dcromp88@googlemail.com"


SCOPES = ['https://www.googleapis.com/auth/calendar']
SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'

In [27]:
from __future__ import print_function

import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account


# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']




def main():
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)


    try:
        service = build('calendar', 'v3', credentials=creds)

        # Call the Calendar API
        now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
        print('Getting the upcoming 10 events')
        events_result = service.events().list(calendarId=googleCalendarId, timeMin=now,
                                              maxResults=10, singleEvents=True,
                                              orderBy='startTime').execute()
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')
            return

        # Prints the start and name of the next 10 events
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(start, event['summary'])

    except HttpError as error:
        print('An error occurred: %s' % error)


if __name__ == '__main__':
    main()



Getting the upcoming 10 events
2023-06-23T19:00:00+02:00 Training
2023-06-23T20:00:00+02:00 Adult Training
2023-06-27T19:00:00+02:00 Training
2023-06-30T19:00:00+02:00 Training
2023-06-30T20:00:00+02:00 Adult Training
2023-07-04T19:00:00+02:00 Training
2023-07-07T19:00:00+02:00 Training
2023-07-07T20:00:00+02:00 Adult Training
2023-07-11T19:00:00+02:00 Training
2023-07-14T19:00:00+02:00 Training


### Add events to Firebase [events] collection

In [38]:
import os
os.environ["GCLOUD_PROJECT"] = "wallet-login-45c1c"

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account


#SCOPES = ['https://www.googleapis.com/auth/calendar']
#SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'

def import_events():
    # Set up the Calendar API
    
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    

    service = build('calendar', 'v3', credentials=creds)


    # Get the events from the public calendar
    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    print('Getting the upcoming 10 events')
        
    events_result = service.events().list(calendarId=googleCalendarId, timeMin=now,
                                              maxResults=10, singleEvents=True,
                                              orderBy='startTime').execute()
    events = events_result.get('items', [])

    #rewrite creds
    if not firebase_admin._apps:

        firebase_cred = credentials.Certificate("../firebase-adminsdk.json")
        default_app = firebase_admin.initialize_app(firebase_cred)

    db = firestore.client()

    # Add each event to Firestore
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(start, event['summary'])

        event_ref = db.collection('events').document(event['summary'])
        event_ref.set(event)
        print(start, event['summary'], ' added')


if __name__ == '__main__':
    import_events()


/home/alexsi/.local/lib/python3.10/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Getting the upcoming 10 events
2023-06-23T19:00:00+02:00 Training
2023-06-23T19:00:00+02:00 Training  added
2023-06-23T20:00:00+02:00 Adult Training
2023-06-23T20:00:00+02:00 Adult Training  added
2023-06-27T19:00:00+02:00 Training
2023-06-27T19:00:00+02:00 Training  added
2023-06-30T19:00:00+02:00 Training
2023-06-30T19:00:00+02:00 Training  added
2023-06-30T20:00:00+02:00 Adult Training
2023-06-30T20:00:00+02:00 Adult Training  added
2023-07-04T19:00:00+02:00 Training
2023-07-04T19:00:00+02:00 Training  added
2023-07-07T19:00:00+02:00 Training
2023-07-07T19:00:00+02:00 Training  added
2023-07-07T20:00:00+02:00 Adult Training
2023-07-07T20:00:00+02:00 Adult Training  added
2023-07-11T19:00:00+02:00 Training
2023-07-11T19:00:00+02:00 Training  added
2023-07-14T19:00:00+02:00 Training
2023-07-14T19:00:00+02:00 Training  added


### Create groups

In [5]:
firestore_client = firestore.client()

In [6]:
# Create a collection named 'groups'
groups_ref = firestore_client.collection('groups')

In [7]:
# Create a document with ID 'group1' and some data
group1_ref = groups_ref.document('group1')
group1_ref.set({
    'name': 'Group 1',
    'members': ['Alice', 'Bob', 'Charlie']
})

# Create another document with ID 'group2' and some data
group2_ref = groups_ref.document('group2')
group2_ref.set({
    'name': 'Group 2',
    'members': ['David', 'Eve', 'Frank']
})

update_time {
  seconds: 1687524363
  nanos: 541386000
}

### Create test events manually
Testing the strucutre

In [36]:
# Create a collection named 'events'
events_ref = firestore_client.collection('events')

# Create a document with ID 'event1' and a reference to group1
event1_ref = events_ref.document('event1')
event1_ref.set({
    'name': 'Event 1',
    'date': '2023-06-30',
    'group': group1_ref # Reference to group1 document
})

# Create another document with ID 'event2' and a reference to group2
event2_ref = events_ref.document('event2')
event2_ref.set({
    'name': 'Event 2',
    'date': '2023-07-01',
    'group': group2_ref # Reference to group2 document
})

update_time {
  seconds: 1687533427
  nanos: 938552000
}

### OLD VERSION OF EVENTS

In [ ]:
# Create a sub-collection for events under the club document
events = club_ref.collection("events")

# Create a document for an event with some data fields
event_id = "456"
event_data = {
    "title": "Fencing Tournament",
    "date": "2023-06-30",
    "location": "Gladius Arena"
}
event_ref = events.document(event_id)
event_ref.set(event_data)

# Create a sub-collection for occurrences under the event document
occurrences = event_ref.collection("occurrences")

# Create a document for an occurrence with some data fields
occurrence_id = "789"
occurrence_data = {
    "start_time": "14:00",
    "end_time": "16:00",
    "participants": ["Alice", "Bob"]
}
occurrence_ref = occurrences.document(occurrence_id)
occurrence_ref.set(occurrence_data)

# Create a sub-collection for attendance under the occurrence document
attendance = occurrence_ref.collection("attendance")

# Create a document for an attendance with some data fields
attendance_id = "1011"
attendance_data = {
    "name": "Alice",
    "score": 3
}
attendance_ref = attendance.document(attendance_id)
attendance_ref.set(attendance_data)

update_time {
  seconds: 1687370617
  nanos: 317941000
}